In [28]:
import pandas as pd
from sklearn import tree
from sklearn.metrics import accuracy_score
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
import itertools
import math
from TMDataset import TMDataset
import const
import util
import numpy as np


In [6]:
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import average_precision_score
from inspect import signature

In [5]:
df = pd.read_csv("TransportationData/datasetBalanced/dataset_5secondWindow.csv")

In [3]:
#df

In [8]:
#df.columns

In [34]:
class TMDetection:
    dataset = TMDataset()
    classes = []
    classes2string = {}
    classes2number = {}

    def __init__(self):
        if not const.HAVE_DT:
            self.dataset.create_balanced_dataset(const.SINTETIC_LEARNING)
        classes_dataset = self.dataset.get_dataset['target'].values
        print(classes_dataset)
        for i, c in enumerate(sorted(set(classes_dataset))):
            self.classes2string[i] = c
            self.classes2number[c] = i
            self.classes.append(c)

    def __get_sets_for_classification(self, df_train, df_test, features):
        train, test = util.fill_nan_with_mean_training(df_train, df_test)
        train_features = train[features].values
        train_classes = [self.classes2number[c] for c in train['target'].values]
        test_features = test[features].values
        test_classes = [self.classes2number[c] for c in test['target'].values]
        return train_features, train_classes, test_features, test_classes

    # random forest algorithm training on training al train set and test on all test set
    def random_forest(self, sensors_set):
        features = list(self.dataset.get_sensors_set_features(sensors_set))
        print("RANDOM FOREST.....")
        print("CLASSIFICATION BASED ON THESE SENSORS: ", self.dataset.get_remained_sensors(sensors_set))
        print("NUMBER OF FEATURES: ", len(features))
        train_features, train_classes, test_features, test_classes = self.__get_sets_for_classification(
            self.dataset.get_train, self.dataset.get_test, features)
        classifier_forest = RandomForestClassifier(n_estimators=const.PAR_RF_ESTIMATOR)
        classifier_forest.fit(train_features, train_classes)
        test_prediction = classifier_forest.predict(test_features)
        
        # metrics
        acc = accuracy_score(test_classes, test_prediction)
        test_pred = test_prediction.tolist()
        print(type(test_classes), type(test_pred))
        print(len(test_classes), len(test_prediction))
        #print(test_classes,test_prediction)
        precision = precision_score(test_classes, test_pred, average='micro')
        print(precision)
        precision = precision_score(test_classes, test_pred, average='macro')
        print(precision)
        precision = precision_score(test_classes, test_pred, average='weighted')
        print(precision)
        f1 = f1_score(test_classes, test_pred, average='weighted')
        #recall = recall_score(test_classes, test_prediction)
        #f1 = f1_score(test_classes, test_prediction)
        
        
        #df_feature = pd.DataFrame(
            #{'accuracy': acc, 'featureName': features, 'importance': classifier_forest.feature_importances_})
        #df_feature = df_feature.sort_values(by='importance', ascending=False)
        #print("ACCURACY : " + str(acc), "Precision:" + str(precision),"recall score:" + str(recall),"F1 score:" + str(f1),)
        
        print("END RANDOM FOREST")

        if not os.path.exists(const.DIR_RESULTS):
            os.makedirs(const.DIR_RESULTS)
        #df_feature.to_csv(const.DIR_RESULTS + "/" + str(sensors_set) + const.FILE_RANDOM_FOREST_RESULTS, index=False)


In [35]:
if __name__ == "__main__":
    detection = TMDetection()
    detection.random_forest(3)




START CREATE BALANCED DATASET....
./TransportationData/_Dataset/dataset.csv
END CREATE BALANCED DATASET....
['Bus' 'Bus' 'Bus' ... 'Walking' 'Walking' 'Walking']
RANDOM FOREST.....
CLASSIFICATION BASED ON THESE SENSORS:  ['android.sensor.game_rotation_vector', 'android.sensor.gyroscope', 'android.sensor.rotation_vector', 'android.sensor.gyroscope_uncalibrated', 'android.sensor.accelerometer', 'android.sensor.linear_acceleration', 'sound', 'android.sensor.orientation', 'speed']
NUMBER OF FEATURES:  36
<class 'list'> <class 'list'>
1833 1833
0.9181669394435352
END RANDOM FOREST


In [ ]:
probs = classifier_forest.predict_proba(test_features)
        
target_probs = probs[:,1]
        fpr, tpr, thresholds = roc_curve(test_classes, target_probs)
        roc_auc = auc(fpr, tpr)
        plt.title('Receiver Operating Characteristic')
        plt.plot(fpr, tpr, 'y', label = 'AUC = %0.2f' % roc_auc)
        plt.legend(loc = 'lower right')
        plt.plot([0, 1], [0, 1],'r--')
        plt.xlim([0, 1])
        plt.ylim([0, 1])
        plt.ylabel('True Positive Rate')
        plt.xlabel('False Positive Rate')
        plt.show()
        

In [25]:
from sklearn.metrics import precision_score
# define actual
act_pos = [1 for _ in range(100)]
act_neg = [0 for _ in range(10000)]
y_true = act_pos + act_neg
# define predictions
pred_pos = [0 for _ in range(10)] + [1 for _ in range(90)]
pred_neg = [1 for _ in range(30)] + [0 for _ in range(9970)]
y_pred = pred_pos + pred_neg
# calculate prediction
precision = precision_score(y_true, y_pred, average='binary')
print('Precision: %.3f' % precision)
print(type(y_true), type(y_pred))
print(len(y_true), len(y_pred))


Precision: 0.750
<class 'list'> <class 'list'>
10100 10100
